In [1]:
from itertools import product
import pandas as pd
import numpy as np
import scpQCA

In [4]:
res = list(product([0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0]))
data=pd.DataFrame(res)
data.columns=['A','B','C','D','E','F','OUTCOME']
data

,A,B,C,D,E,F,OUTCOME
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0
3,0,0,0,0,1,1,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
59,1,1,1,0,1,1,0
60,1,1,1,1,0,0,0
61,1,1,1,1,0,1,0
62,1,1,1,1,1,0,0


In [5]:
# A=0 * B=0 + C=1 * D=1 + A=0 * C=0 * E=0 + B=1 * D=1 * F=1 -> OUTCOME=1
for i in range(len(data)):
    if (data.iloc[i]['A']==0 and data.iloc[i]['B']==0) or \
        (data.iloc[i]['C']==1 and data.iloc[i]['D']==1) or \
        (data.iloc[i]['A']==0 and data.iloc[i]['C']==0 and data.iloc[i]['E']==0) or \
        (data.iloc[i]['B']==1 and data.iloc[i]['D']==1 and data.iloc[i]['F']==1):
        data.iloc[i]['OUTCOME']=1
data

,A,B,C,D,E,F,OUTCOME
0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,1
2,0,0,0,0,1,0,1
3,0,0,0,0,1,1,1
4,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...
59,1,1,1,0,1,1,0
60,1,1,1,1,0,0,1
61,1,1,1,1,0,1,1
62,1,1,1,1,1,0,1


In [6]:
data.sum(axis=0)

A          32
B          32
C          32
D          32
E          32
F          32
OUTCOME    35
dtype: int64

In [108]:
# data.query('A==0 & B==1 & OUTCOME==1')

,A,B,C,D,E,F,OUTCOME
28,0,1,1,1,0,0,1
29,0,1,1,1,0,1,1
30,0,1,1,1,1,0,1
31,0,1,1,1,1,1,1


In [48]:
temp=data.sample(n=200,replace=True,axis=0)
temp

,A,B,C,D,E,F,OUTCOME
45,1,0,1,1,0,1,1
30,0,1,1,1,1,0,1
50,1,1,0,0,1,0,0
51,1,1,0,0,1,1,0
60,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...
6,0,0,0,1,1,0,1
15,0,0,1,1,1,1,1
37,1,0,0,1,0,1,0
55,1,1,0,1,1,1,1


In [49]:
temp.sum(axis=0)

A          103
B           94
C           77
D          104
E          103
F           99
OUTCOME    106
dtype: int64

## 添加混淆项

In [50]:
index = np.random.choice(np.arange(len(temp)), size=15, replace=False)
for i in range(len(temp)):
    if i in index:
        print(temp.iloc[i])
        temp.iloc[i]['OUTCOME']=1 if temp.iloc[i]['OUTCOME']==0 else 0
        print(temp.iloc[i])

A          1
B          1
C          0
D          0
E          1
F          1
OUTCOME    0
Name: 51, dtype: int64
A          1
B          1
C          0
D          0
E          1
F          1
OUTCOME    1
Name: 51, dtype: int64
A          0
B          0
C          1
D          1
E          1
F          1
OUTCOME    1
Name: 15, dtype: int64
A          0
B          0
C          1
D          1
E          1
F          1
OUTCOME    0
Name: 15, dtype: int64
A          0
B          0
C          0
D          1
E          1
F          0
OUTCOME    1
Name: 6, dtype: int64
A          0
B          0
C          0
D          1
E          1
F          0
OUTCOME    0
Name: 6, dtype: int64
A          1
B          1
C          0
D          0
E          0
F          0
OUTCOME    0
Name: 48, dtype: int64
A          1
B          1
C          0
D          0
E          0
F          0
OUTCOME    1
Name: 48, dtype: int64
A          1
B          1
C          0
D          1
E          0
F          1
OUTCOME    1

In [51]:
temp.sum(axis=0)

A          103
B           94
C           77
D          104
E          103
F           99
OUTCOME    105
dtype: int64

In [52]:
temp.to_csv("D:\code\scpQCA\paper code\highcover\highcover6.csv",index=False)

In [11]:
import scpQCA
feature_list=['A','B','C','D','E','F','OUTCOME']
scp=pd.read_csv(r'D:\code\scpQCA\paper code\highcover\highcover5.csv')
scp

,A,B,C,D,E,F,OUTCOME
0,0,1,1,0,1,0,0
1,0,0,0,0,0,1,1
2,0,0,1,0,1,1,1
3,0,1,0,1,0,0,1
4,0,1,1,1,0,1,1
...,...,...,...,...,...,...,...
195,0,1,1,1,0,0,1
196,0,0,0,1,1,0,1
197,1,0,1,1,1,0,1
198,0,0,0,1,1,1,1


In [6]:
test=scpQCA.scpQCA(scp,decision_name='OUTCOME',caseid='')
test.search_necessity(decision_label=1,feature_list=feature_list,consistency_threshold=0.9)

In [7]:
rules=test.candidate_rules(decision_label=1,feature_list=feature_list,consistency=0.9,cutoff=10,rule_length=4)
test.scp_truth_table(rules=rules,feature_list=feature_list,decision_label=1)

Running...please wait. There are 57 factor combinations.
There are 34 candidate rules in total.
    A  B  C  D  E  F  number consistency coverage
0   0  0  -  -  -  -      48      0.9167   0.4314
1   0  0  -  -  1  -      31      0.9032   0.2745
2   -  1  1  1  -  -      28      0.9286   0.2549
3   -  -  1  1  -  0      25      0.9600   0.2353
4   0  0  0  -  -  -      22      0.9545   0.2059
5   0  0  -  -  -  0      20      1.0000   0.1961
6   0  0  -  1  -  -      20      0.9500   0.1863
7   0  -  -  1  0  -      19      0.9474   0.1765
8   0  -  0  -  -  1      20      0.9000   0.1765
9   0  1  1  1  -  -      20      0.9000   0.1765
10  -  -  1  1  1  0      17      0.9412   0.1569
11  -  1  1  1  1  -      17      0.9412   0.1569
12  -  -  1  1  0  -      17      0.9412   0.1569
13  0  0  -  -  0  -      17      0.9412   0.1569
14  -  1  1  1  -  0      15      1.0000   0.1471
15  0  -  1  1  -  0      13      1.0000   0.1275
16  -  1  -  1  1  1      14      0.9286   0.1275
17  

,A,B,C,D,E,F,number,consistency,coverage
0,0,0,-,-,-,-,48,0.9167,0.4314
1,0,0,-,-,1,-,31,0.9032,0.2745
2,-,1,1,1,-,-,28,0.9286,0.2549
3,-,-,1,1,-,0,25,0.9600,0.2353
4,0,0,0,-,-,-,22,0.9545,0.2059
5,0,0,-,-,-,0,20,1.0000,0.1961
6,0,0,-,1,-,-,20,0.9500,0.1863
7,0,-,-,1,0,-,19,0.9474,0.1765
8,0,-,0,-,-,1,20,0.9000,0.1765
9,0,1,1,1,-,-,20,0.9000,0.1765


In [9]:
config,issue=test.greedy(rules=rules,decision_label=1,unique_cover=5)
config,issue

(['B==0 & A==0', 'D==1 & C==1 & B==1', 'F==1 & E==1 & D==1 & B==1'],
 {1,
  2,
  4,
  8,
  9,
  10,
  11,
  15,
  17,
  22,
  23,
  25,
  29,
  30,
  32,
  33,
  34,
  40,
  50,
  51,
  52,
  56,
  59,
  60,
  61,
  64,
  66,
  72,
  73,
  75,
  77,
  87,
  90,
  91,
  92,
  93,
  94,
  96,
  99,
  100,
  106,
  109,
  111,
  113,
  120,
  124,
  125,
  126,
  134,
  135,
  136,
  137,
  139,
  147,
  150,
  151,
  152,
  153,
  154,
  155,
  157,
  162,
  164,
  172,
  174,
  175,
  179,
  180,
  183,
  185,
  187,
  193,
  194,
  195,
  196,
  198,
  199})

In [10]:
test.cov_n_con(decision_label=1, configuration=config, issue_sets=issue)
print(config)

consistency = 0.927710843373494 and coverage = 0.7549019607843137
['B==0 & A==0', 'D==1 & C==1 & B==1', 'F==1 & E==1 & D==1 & B==1']


In [197]:
scp.sum(axis=0)

A           84
B          106
C          106
D           92
E          102
F           95
OUTCOME     88
dtype: int64

In [73]:
scp=pd.read_csv(r'D:\code\scpQCA\paper code\highcover\highcover5.csv')

In [74]:
# highcover1_eQMC: ab + CD + ace + BDF
# config1='A==0 & B==0'
# config2='C==1 & D==1'
# config3='A==0 & C==0 & E==0'
# config4='B==1 & D==1 & F==1'
# highcover2_eQMC: ab + ACD + aDe + BDF
# config1='A==0 & B==0'
# config2='A==0 & C==1 & D==1'
# config3='A==0 & D==1 & E==0'
# config4='B==1 & D==1 & F==1'
# highcover3_eQMC: ab + CD + BDF + acde
# config1='A==0 & B==0'
# config2='C==1 & D==1'
# config3='A==0 & C==0 & D==1 & E==0'
# config4='B==1 & D==1 & F==1'
# highcover4_eQMC: ab + aCD + aCE + bCD + BDF + acdef
# config1='A==0 & B==0'
# config2='A==0 & C==1 & D==1'
# config3='A==0 & C==1 & E==1'
# config4='B==0 & C==1 & D==1'
# config5='B==1 & D==1 & F==1'
# config6='A==0 & C==0 & D==0 & E==0 & F==0'
# highcover6_eQMC: ACD + ace + BDEF + (CDf + aBDF + abeF + abEf + acDF)
# config1='A==1 & C==1 & D==1'
# config2='A==0 & C==0 & E==0'
# config3='B==1 & D==1 & E==1 & F==1'
# config4='F==0 & C==1 & D==1'
# config5='A==0 & B==1 & D==1 & F==1'
# config6='A==0 & B==0 & E==0 & F==1'
# config7='A==0 & B==0 & E==1 & F==0'
# config8='A==0 & C==0 & D==1 & F==1'
# highcover5_eQMC: abf + aDe + CDe + BDEF + ABCEf + (abCd + abcE + aCDf + BcdeF)
config1='A==0 & B==0 & F==0'
config2='A==0 & D==1 & E==0'
config3='C==1 & D==1 & E==0'
config4='B==1 & D==1 & E==1 & F==1'
config5='A==1 & B==1 & C==1 & E==1 & F==0'
config9='A==0 & B==0 & C==1 & D==0'
config6='A==0 & B==0 & C==0 & E==1'
config7='A==0 & C==1 & D==1 & F==0'
config8='B==1 & C==0 & D==0 & E==0 & F==1'


config=[config1,config2,config3,config4,config5,config6,config7,config8,config9]
issue=set()
for c in config:
    temp=scp.query(c)
    cset=set(list(temp[temp['OUTCOME']==1].index))
    # cset=set(list(data.query(c).index))
    issue=issue.union(cset)
    others=set()
    for i in config:
        if i != c:
            others=others.union(set(list(data.query(i).index)))
    print(len(cset))
    print(others)
    print(cset-others)
    print()
print(len(issue))
test=scpQCA.scpQCA(scp,decision_name='OUTCOME',caseid='')
test.cov_n_con(decision_label=1, configuration=config, issue_sets=issue)

20
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21, 23, 28, 29, 30, 31, 44, 45, 49, 55, 58, 60, 61, 62, 63}
{32, 64, 194, 185, 196, 134, 73, 174, 51, 179, 152, 153, 154, 91, 157, 94}

18
{0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 23, 28, 29, 30, 31, 44, 45, 49, 55, 58, 60, 61, 62, 63}
{194, 163, 195, 106, 139, 109, 147, 180, 22, 120, 25, 92, 152}

16
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21, 23, 28, 29, 30, 31, 49, 55, 58, 62, 63}
{194, 195, 164, 106, 139, 109, 178, 147, 22, 86, 25, 92, 93}

13
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21, 28, 29, 30, 44, 45, 49, 58, 60, 61, 62}
{33, 162, 100, 135, 72, 136, 137, 75, 77, 59, 124}

6
{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21, 23, 28, 29, 30, 31, 44, 45, 49, 55, 60, 61, 63}
{165, 40, 111, 52, 183, 126}

11
{0, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 17, 20, 21, 23, 28, 29, 30, 31, 44, 45, 49, 55, 58, 60, 61, 62, 63}
{34, 99, 196, 198, 134, 73, 179, 56, 154, 125}

13
{0, 2, 3, 4,

0.7371840302386015

In [220]:
# config1='A==0 & B==0'
# config2='B==1 & C==1 & D==1'
# config3='C==1 & D==1 & E==0'
# config4='C==1 & D==1 & F==1'
config1='C==1 & D==1'
config2='A==0 & B==0 & C==1'
config3='A==0 & B==0 & D==1'
config4='A==0 & B==0 & E==1'
config5='A==0 & C==0 & D==0 & E==0 & F==1'
# config1='A==0 & B==0 & C==1'
# config2='A==0 & B==0 & D==1'
# config3='A==0 & B==0 & E==0'
# config4='C==1 & D==1 & E==0'
# config5='C==1 & D==1 & F==1'
# config6='B==1 & C==1 & D==1'
# config7='B==0 & C==0 & D==0 & E==1 & F==1'
# config1='A==0 & B==0'
# config2='A==0 & C==1 & D==1'
# config3='B==0 & C==1 & D==1'
# config4='C==1 & D==1 & F==1'
# config5='A==1 & B==1 & C==1 & E==0 & F==0'
# config1='A==0 & B==0 & D==1'
# config2='A==1 & B==1 & C==1 & D==1'
# config3='A==0 & B==0 & E==0 & F==1'
# config4='A==0 & B==0 & E==1 & F==0'
# config5='B==0 & C==0 & D==1 & E==1 & F==0'
# config6='A==0 & B==0 & C==0 & F==1'
# config7='B==0 & C==1 & D==1 & E==0 & F==0'
# config8='B==1 & C==1 & D==1 & E==0 & F==1'
# config9='B==1 & C==1 & D==1 & E==1 & F==0'
config=[config1,config2,config3,config4,config5]
issue=set()
for c in config:
    temp=scp.query(c)
    cset=set(list(temp[temp['OUTCOME']==1].index))
    # cset=set(list(data.query(c).index))
    issue=issue.union(cset)
    others=set()
    for i in config:
        if i != c:
            others=others.union(set(list(data.query(i).index)))
    print(len(cset))
    print(others)
    print(cset-others)
    print()
print(len(issue))
test=scpQCA.scpQCA(scp,decision_name='OUTCOME',caseid='')
test.cov_n_con(decision_label=1, configuration=config, issue_sets=issue)

56
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17}
{0, 133, 134, 136, 137, 144, 20, 149, 22, 23, 24, 27, 29, 30, 159, 160, 161, 162, 38, 172, 173, 48, 49, 176, 51, 177, 180, 55, 58, 59, 60, 62, 65, 66, 69, 75, 77, 86, 87, 88, 89, 92, 98, 109, 113, 123, 126}

25
{1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 28, 29, 30, 31, 44, 45, 46, 47, 60, 61, 62, 63}
{147, 148, 24, 152, 27, 158, 160, 34, 42, 51, 180, 188, 66, 68, 88, 110, 119, 120, 123, 126}

31
{1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 28, 29, 30, 31, 44, 45, 46, 47, 60, 61, 62, 63}
{130, 132, 4, 5, 140, 24, 27, 156, 160, 35, 165, 51, 180, 185, 187, 66, 78, 80, 88, 97, 101, 111, 112, 114, 121, 123, 126}

25
{1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 28, 29, 30, 31, 44, 45, 46, 47, 60, 61, 62, 63}
{129, 130, 140, 147, 148, 152, 24, 27, 156, 36, 165, 167, 168, 42, 180, 187, 78, 80, 101, 110, 114, 120, 123, 125}

5
{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 28, 29, 30, 31, 44, 45, 46, 47, 60, 61, 62, 6

0.9406961314688169